In [5]:
import numpy as np
import pandas as pd

shops = pd.read_csv('input/shops.csv', index_col=1)
shops.head()

,shop_name
shop_id,
0,"!Якутск Орджоникидзе, 56 фран"
1,"!Якутск ТЦ ""Центральный"" фран"
2,"Адыгея ТЦ ""Мега"""
3,"Балашиха ТРК ""Октябрь-Киномир"""
4,"Волжский ТЦ ""Волга Молл"""


In [6]:
categs = pd.read_csv('input/item_categories.csv', index_col=1)
categs.head()

,item_category_name
item_category_id,
0,PC - Гарнитуры/Наушники
1,Аксессуары - PS2
2,Аксессуары - PS3
3,Аксессуары - PS4
4,Аксессуары - PSP


In [7]:
items = pd.read_csv('input/items.csv', index_col=1)
items.head()

,item_name,item_category_id
item_id,,
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,40
1,!ABBYY FineReader 12 Professional Edition Full...,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,40
4,***КОРОБКА (СТЕКЛО) D,40


In [9]:
def get_categ_id_for_item(item_id):
    return items.loc[item_id]['item_category_id']

get_categ_id_for_item(1)

76

In [11]:
transactions = pd.read_csv('input/sales_train_v2.csv')
print('shape:', transactions.shape)
print('max date_block_num:', transactions['date_block_num'].max())
transactions.head()

shape: (2935849, 6)
max date_block_num: 33


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [12]:
test_data = pd.read_csv('input/test.csv', index_col=0)
print(test_data.shape)
test_data.head()

(214200, 2)


,shop_id,item_id
ID,,
0,5,5037
1,5,5320
2,5,5233
3,5,5232
4,5,5268


In [13]:
monthly_totals = transactions.groupby(['date_block_num', 'shop_id', 'item_id'],as_index=False)[['item_cnt_day']].sum()
print(monthly_totals.shape)
monthly_totals.head()

(1609124, 4)


,date_block_num,shop_id,item_id,item_cnt_day
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0


In [14]:
#save file
monthly_totals.to_csv('input/monthly_totals.csv', index=False)

In [16]:
#add category id
monthly_totals = monthly_totals.set_index('item_id').join(items).drop('item_name', axis=1).reset_index()

In [18]:
monthly_totals.to_csv('input/monthly_totals_categ.csv', index=False)
monthly_totals.head()

,item_id,date_block_num,shop_id,item_cnt_day,item_category_id
0,0,20,54,1.0,40
1,1,15,55,2.0,76
2,1,18,55,1.0,76
3,1,19,55,1.0,76
4,1,20,55,1.0,76


In [19]:
#change date_block_num
def get_year_from_date_block_num(date_block_num):
    if date_block_num < 12:
        return 2013
    elif date_block_num < 24:
        return 2014
    else:
        return 2015

def get_month_from_date_block_num(date_block_num):
    return int(date_block_num % 12)

print('date block: 33 =' , get_year_from_date_block_num(33), get_month_from_date_block_num(33))


date block: 33 = 2015 9


In [20]:
def get_year_for_row(row):
    return get_year_from_date_block_num(row['date_block_num'])

monthly_totals['year'] = monthly_totals.apply(lambda row: get_year_for_row(row), axis=1)
monthly_totals.head()

,item_id,date_block_num,shop_id,item_cnt_day,item_category_id,year
0,0,20,54,1.0,40,2014
1,1,15,55,2.0,76,2014
2,1,18,55,1.0,76,2014
3,1,19,55,1.0,76,2014
4,1,20,55,1.0,76,2014


In [21]:
def get_month_for_row(row):
    return get_month_from_date_block_num(row['date_block_num'])

monthly_totals['month'] = monthly_totals.apply(lambda row: get_month_for_row(row), axis=1)
monthly_totals.head()

,item_id,date_block_num,shop_id,item_cnt_day,item_category_id,year,month
0,0,20,54,1.0,40,2014,8
1,1,15,55,2.0,76,2014,3
2,1,18,55,1.0,76,2014,6
3,1,19,55,1.0,76,2014,7
4,1,20,55,1.0,76,2014,8


In [52]:
#monthly_totals.to_csv('input/monthly_totals_categ_date.csv', index=False)
monthly_totals = pd.read_csv('input/monthly_totals_categ_date.csv')
monthly_totals.head()

,item_id,date_block_num,shop_id,item_cnt_day,item_category_id,year,month
0,0,20,54,1.0,40,2014,8
1,1,15,55,2.0,76,2014,3
2,1,18,55,1.0,76,2014,6
3,1,19,55,1.0,76,2014,7
4,1,20,55,1.0,76,2014,8


In [53]:
#monthly_totals.reset_index()
#monthly_totals.set_index(['date_block_num','shop_id','item_id'], inplace=True)
indexed_monthly_totals = monthly_totals.set_index(['date_block_num','shop_id','item_id'])
indexed_monthly_totals.head()

,,,item_cnt_day,item_category_id,year,month
date_block_num,shop_id,item_id,,,,
20,54,0,1.0,40,2014,8
15,55,1,2.0,76,2014,3
18,55,1,1.0,76,2014,6
19,55,1,1.0,76,2014,7
20,55,1,1.0,76,2014,8


In [82]:
#indexed_monthly_totals.loc[(20, 54, 0)]['item_cnt_day']
#if indexed_monthly_totals.index.isin([(19, 55, 1)]).any():
#    print("in")
#else:
#    print("not in")

#indexed_monthly_totals.index.values
indexed_monthly_totals.index.values.isin([(15,55,1)])

AttributeError: 'numpy.ndarray' object has no attribute 'isin'

In [71]:
def get_total_for_prev_month(df, date_block_num, shop_id, item_id, prev_month_index):
    target_date_block_num = date_block_num + prev_month_index
    if target_date_block_num < 0:
        return np.nan
    elif target_date_block_num > 33:
        return np.nan
    
    #df = monthly_totals
    #found = df.loc[(df['date_block_num'] == target_date_block_num) & (df['shop_id'] == shop_id) & (df['item_id'] == item_id)]
    #if found.empty:
    idx = (int(target_date_block_num), int(shop_id), int(item_id)) 
    if idx in df.columns:
        return df.loc[idx]['item_cnt_day']
    else:
        return 0
    
total = get_total_for_prev_month(indexed_monthly_totals, 20, 55, 1, -1)
print(total)

0


In [63]:
def get_prev_month_total_for_row(row, df, prev_month_index):
    return get_total_for_prev_month(df, row['date_block_num'],row['shop_id'],row['item_id'],prev_month_index)

monthly_totals['item_cnt_tm1'] = monthly_totals.apply(lambda row: get_prev_month_total_for_row(row, indexed_monthly_totals, -1), axis=1)
#training_data['item_cnt_tm2'] = training_data.apply(lambda row: get_prev_month_total_for_row(row, -2), axis=1)
#training_data['item_cnt_tm12'] = training_data.apply(lambda row: get_prev_month_total_for_row(row, -12), axis=1)
#training_data['item_cnt_tp1'] = training_data.apply(lambda row: get_prev_month_total_for_row(row, 1), axis=1)
#training_data.loc[training_data['item_cnt_tp1'] > 0.0]

KeyError: ((19, 54, 0), 'occurred at index 0')

In [20]:
#divide into smaller files
monthly_totals_0 = monthly_totals.loc[(monthly_totals['shop_id'] >= 0) & (monthly_totals['shop_id'] < 10)]
print(monthly_totals_0.shape)
monthly_totals_0.head()

(172338, 4)


,date_block_num,shop_id,item_id,item_cnt_day
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0


In [38]:
training_data = pd.read_csv('input/monthly_totals_0.csv')
training_data.head()

,date_block_num,shop_id,item_id,item_cnt_day
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0


In [48]:
#drop rows with nan
training_data.dropna(inplace=True)
print(training_data.shape)

(92390, 9)


In [7]:
training_data.drop('date_block_num', axis=1)
training_data.to_csv('input/training_data_0_categ_date.csv', index=False)